In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import json, os, PIL
from PIL import Image
from collections import defaultdict

In [2]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
import keras.backend as K

Using TensorFlow backend.


In [3]:
def limit_mem():
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

limit_mem()

In [4]:
model = InceptionResNetV2(include_top=False, pooling='avg', input_shape=(224,224,3))

In [12]:
fpath = '../../Downloads/mscoco/train2017/'

In [5]:
with open('train_seen_img_box_cat.json', 'r') as f:
    img_box_cat = json.load(f)

In [6]:
img_box_cat_ = {}
for item in img_box_cat:
    l= list(item.keys())[0]
    img_box_cat_[l] = item[l]

In [19]:
a = 0

In [ ]:
im = Image.open(fpath+ os.listdir(fpath)[a]); a+=1; im

bbox = img_box_cat[0]['000000391895.jpg'][1]['bbox:']; bbox

coord = (bbox[0], bbox[1], bbox[2]+bbox[0], bbox[3]+bbox[1])

img = Image.open(fpath+'000000391895.jpg')

img.crop(coord).resize((224,224))

res = model.predict_on_batch(preprocess_input(np.array(img.crop(coord).resize((224,224)), dtype='float32').reshape(-1, 224,224,3)))

res.shape

In [7]:
with open('color_images.json', 'r') as f:
    final_list = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'color_images.json'

In [ ]:
len(final_list)

In [ ]:
desired_list = []

In [ ]:
for i in final_list:
    k = list(img_box_cat[i].keys())[0]
    v = img_box_cat[i][k]
    for item in v:
        a = [k, item]
        desired_list.append(a)

In [ ]:
print(len(desired_list), desired_list[0])

final_list = []

for i in range(0, len(img_box_cat), 10000):
    with ThreadPoolExecutor(max_workers=128) as execr:
        res = execr.map(get_size, range(i, min(i+10000, n)))
        for r in res: app_(r)
    print(i, len(final_list))

def get_size(i):
    it = img_box_cat[i]
    img = Image.open(fpath+list(it.keys())[0])
    img_array = np.array(img, dtype='uint8')
    return (i, img_array.shape)

def app_(ip):
    shp = ip[1]
    global final_list
    if len(shp) ==3:
        if shp[2] == 3:
            final_list.append(ip[0])

In [2]:
pwd

'/home/met/ug/16144001/ml_18/Zero-Shot-Object-Detection-Segmentation'

In [3]:
import threading, bcolz
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
# from bcolz_array_iterator import BcolzArrayIterator

In [8]:
bc_path = '/home/met/ug/16144001/ml_18/Zero-Shot-Object-Detection-Segmentation/coco_seen_train_features.bc'

In [ ]:
# arr = bcolz.carray(np.empty((0, 1536), 'float32'), 
#                    chunklen=16, mode='r', rootdir=bc_path)

In [5]:
arr = bcolz.open(bc_path)

In [7]:
arr[0]

array([0.16688175, 0.36298326, 0.37069005, ..., 0.32237312, 1.3802015 ,
       0.5637785 ], dtype=float32)

In [43]:
# tl = threading.local()
# tl.place = np.zeros((128,1536), 'float32')
# tl.place = np.zeros((224,224,3), 'uint8')
n = len(desired_list); print(n)
img_batch = []
all_cat_names = []
# total_predictions = np.zeros((n,1536), dtype='float32')
step=4096

663866


In [9]:
def process_img(i):
    value = desired_list[i]
    img = Image.open(fpath+value[0])
    item = value[1]
    bbox = item['bbox:']
    cat_name = item['category']
    coord = (bbox[0], bbox[1], bbox[2]+bbox[0], bbox[3]+bbox[1])
    img_array = np.array(img.crop(coord).resize((224,224)), dtype='float32')
#     if img_array.shape == (224,224,3):
    return (cat_name, img_array)
#     else: print('couldn\'t resize the image', img_array.shape)

def app_img(r):
    all_cat_names.append(r[0])
#     tl.place[:] = (np.array(r[1]) if r else 0)
    img_batch.append(r[1])
    
def predict():    
    global img_batch
#     print(len(img_batch))
    arr.append(model.predict( preprocess_input(np.stack(img_batch)) , batch_size=128) )
    img_batch=[]
#     if len(img_batch) != 0: print('couldn\'t clear img_batch')

In [11]:
plt.imshow(process_img(1).astype('uint8'))

NameError: name 'desired_list' is not defined

In [48]:
for i in tqdm(range(344064, n, step)):
    with ThreadPoolExecutor(max_workers=64) as execr:
        res = execr.map(process_img, range(i, min(i+step, n)))
        for r in res: app_img(r)
    predict()
    arr.flush()
        
with open('all_cat_names.json', 'w') as f:
    json.dump(all_cat_names, f)



  0%|          | 0/79 [00:00<?, ?it/s]

  1%|▏         | 1/79 [09:10<11:55:50, 550.65s/it]

  3%|▎         | 2/79 [12:13<7:50:35, 366.70s/it] 

  4%|▍         | 3/79 [15:17<6:27:14, 305.72s/it]

  5%|▌         | 4/79 [18:21<5:44:09, 275.32s/it]

  6%|▋         | 5/79 [21:25<5:16:59, 257.02s/it]

  8%|▊         | 6/79 [24:27<4:57:38, 244.64s/it]

  9%|▉         | 7/79 [27:29<4:42:50, 235.71s/it]

 10%|█         | 8/79 [30:32<4:31:02, 229.05s/it]

 11%|█▏        | 9/79 [33:35<4:21:13, 223.91s/it]

 13%|█▎        | 10/79 [36:39<4:12:56, 219.95s/it]

 14%|█▍        | 11/79 [39:48<4:06:05, 217.13s/it]

 15%|█▌        | 12/79 [42:58<3:59:58, 214.91s/it]

 16%|█▋        | 13/79 [46:11<3:54:28, 213.16s/it]

 18%|█▊        | 14/79 [49:23<3:49:19, 211.68s/it]

 19%|█▉        | 15/79 [52:34<3:44:19, 210.31s/it]

 20%|██        | 16/79 [55:46<3:39:35, 209.14s/it]

 22%|██▏       | 17/79 [58:56<3:34:59, 208.05s/it]

 23%|██▎       | 18/79 [1:02:00<3:30:08, 206.70s/it]

 24%|██▍       | 19/79 [1:0

In [ ]:
for i in tqdm(range(0, n, step)):
#     with ThreadPoolExecutor(max_workers=64) as execr:
#         res = execr.map(process_img, range(i, min(i+step, n)))
#         for r in res: app_img(r)
#     predict()
#     arr.flush()
        
# with open('all_cat_names.json', 'w') as f:
#     json.dump(all_cat_names, f)

 48%|████▊     | 78/163 [3:49:06<4:09:39, 176.23s/it]

no_of_boxes = []
for i in img_box_cat[:10]:
    name = list(i.keys())[0]
    no_of_boxes.append(len(i[name]))

np.median(no_of_boxes)

for i in img_box_cat[:10]:
    name = list(i.keys())[0]
    print(len(i[name]), end=' ')

import time
from tqdm import tqdm

for i in tqdm(range(10)):
    time.sleep(0.1)

## Checking Error

## Still there is error size of total objects does not match the length of the array

In [42]:
del value, img, item, bbox, cat_name, coord, img_array

In [23]:
first_stored_prob = arr[0]

In [29]:
value = desired_list[344063]
img = Image.open(fpath+value[0])
item = value[1]
bbox = item['bbox:']
cat_name = item['category']
coord = (bbox[0], bbox[1], bbox[2]+bbox[0], bbox[3]+bbox[1])
img_array = np.array(img.crop(coord).resize((224,224)), dtype='float32')
#     if img_array.shape == (224,224,3):
prob = model.predict( preprocess_input(img_array.reshape(-1,224,224,3)))

In [ ]:
663866

In [49]:
arr

carray((672058, 1536), float32)
  nbytes := 3.85 GB; cbytes := 3.57 GB; ratio: 1.08
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  chunklen := 16; chunksize: 98304; blocksize: 98304
  rootdir := '/home/met/ug/16144001/ml_18/Zero-Shot-Object-Detection-Segmentation/coco_seen_train_features.bc'
  mode    := 'a'
[[  1.66881755e-01   3.62983257e-01   3.70690048e-01 ...,   3.22373122e-01
    1.38020146e+00   5.63778520e-01]
 [  1.51414052e-01   6.24765396e-01   2.21562177e-01 ...,   2.28613243e-01
    2.47435403e+00   1.16587436e+00]
 [  7.77609348e-02   9.21879113e-01   1.12419069e+00 ...,   3.73364717e-01
    1.24330533e+00   1.69906902e+00]
 ..., 
 [  4.66625750e-01   3.00940812e-01   6.81944609e-01 ...,   2.24790826e-01
    6.74494624e-01   1.22247541e+00]
 [  1.17359512e-01   4.13591206e-01   3.26162457e-01 ...,   6.00852907e-01
    1.39923140e-01   1.59403408e+00]
 [  2.38941070e-02   1.22237206e-03   0.00000000e+00 ...,   4.59886044e-01
    1.01804495e+00   2.893

In [26]:
first_stored_prob

array([0.16688175, 0.36298326, 0.37069005, ..., 0.32237312, 1.3802015 ,
       0.5637785 ], dtype=float32)

In [28]:
last_stored_prob = arr[344063]; last_stored_prob

array([0.2640972 , 0.1215348 , 0.78102845, ..., 0.51005375, 0.33493176,
       0.5856374 ], dtype=float32)

In [30]:
prob

array([[0.26409733, 0.12153473, 0.7810257 , ..., 0.5100514 , 0.33493188,
        0.5856369 ]], dtype=float32)